# QCNN medMNIST Pipeline

This notebook runs the complete preprocessing, training, and reporting workflow for the `src` package.

In [ ]:
from __future__ import annotations

import os
import subprocess
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
os.environ.setdefault('QCNN_PROJECT_ROOT', str(PROJECT_ROOT))

print(f'Project root: {PROJECT_ROOT}')

## Experiment Configuration

Set dataset and runtime parameters.

In [ ]:
dataset_name = 'bloodmnist'
stride = 3
epochs = 2
batch_size = 32
seed = 123

DATA_DIR = PROJECT_ROOT / 'data'
LOGS_DIR = PROJECT_ROOT / 'logs'
REPORTS_DIR = PROJECT_ROOT / 'reports'

## Execute Preprocessing and Training

The commands below run the package CLIs from the current Python environment.

In [ ]:
def run_command(args: list[str]) -> None:
    print(' '.join(args))
    completed = subprocess.run(args, cwd=PROJECT_ROOT, check=False)
    if completed.returncode != 0:
        raise RuntimeError(f'Command failed with exit code {completed.returncode}: {args}')

run_command([
    sys.executable,
    '-m',
    'src.scripts.preprocess',
    '--dataset', dataset_name,
    '--batch', str(batch_size),
    '--epochs', '2',
    '--seed', str(seed),
    '--device', 'cpu',
    '--data-dir', str(DATA_DIR),
])

run_command([
    sys.executable,
    '-m',
    'src.scripts.train',
    '--dataset', dataset_name,
    '--epochs', str(epochs),
    '--batch', str(batch_size),
    '--stride', str(stride),
    '--seed', str(seed),
    '--data-dir', str(DATA_DIR),
    '--logs-dir', str(LOGS_DIR),
])

## Generate Reports

These commands create prediction tables, confusion matrix, activation heatmap, and learning curves.

In [ ]:
run_dirs = sorted(LOGS_DIR.glob(f'{dataset_name}_run_*'))
if not run_dirs:
    raise RuntimeError('No run directory found after training.')

run_dir = run_dirs[-1]
print(f'Using run directory: {run_dir}')

run_command([
    sys.executable,
    '-m',
    'src.scripts.report',
    '--dataset', dataset_name,
    '--logdir', str(run_dir),
    '--stride', str(stride),
    '--data-dir', str(DATA_DIR),
    '--logs-dir', str(LOGS_DIR),
    '--reports-dir', str(REPORTS_DIR),
])

run_command([
    sys.executable,
    '-m',
    'src.scripts.heatmap',
    '--dataset', dataset_name,
    '--logdir', str(run_dir),
    '--idx', '0',
    '--stride', str(stride),
    '--data-dir', str(DATA_DIR),
    '--logs-dir', str(LOGS_DIR),
    '--reports-dir', str(REPORTS_DIR),
])

run_command([
    sys.executable,
    '-m',
    'src.scripts.plot_curves',
    '--logdir', str(run_dir),
    '--logs-dir', str(LOGS_DIR),
    '--reports-dir', str(REPORTS_DIR),
])

## Display Generated Figures

In [ ]:
import matplotlib.pyplot as plt

figure_dir = REPORTS_DIR / run_dir.name / 'figures'
for image_path in sorted(figure_dir.glob('*.png')):
    image = plt.imread(image_path)
    plt.figure(figsize=(6, 5))
    plt.imshow(image)
    plt.axis('off')
    plt.title(image_path.name)
    plt.show()